In [1]:
# import imageio
# imageio.plugins.ffmpeg.download()
import moviepy.editor as mpe
# from IPython.display import display
from glob import glob

RuntimeError: imageio.ffmpeg.download() has been deprecated. Use 'pip install imageio-ffmpeg' instead.'

In [3]:
import sys, os
import numpy as np
import scipy

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
# MAX_ITERS = 10
TOL = 1.0e-8

In [4]:
# Robus PCA; Principal Component Pursuit (PCP)
from scipy import sparse
from sklearn.utils.extmath import randomized_svd

In [5]:
TOL=1e-9
MAX_ITERS=3

In [6]:
def converged(Z, d_norm):
    err = np.linalg.norm(Z, 'fro') / d_norm
    print('error: ', err)
    return err < TOL

In [7]:
def shrink(M, tau):
    S = np.abs(M) - tau
    return np.sign(M) * np.where(S>0, S, 0)

In [8]:
def _svd(M, rank): return randomized_svd(M, k=min(rank, np.min(M.shape)), raw=True)

In [9]:
def norm_op(M): return _svd(M, 1)[1][0]

In [10]:
def svd_reconstruct(M, rank, min_sv):
    u, s, v = _svd(M, rank)
    s -= min_sv
    nnz = (s > 0).sum()
    return u[:,:nnz] @ np.diag(s[:nnz]) @ v[:nnz], nnz

In [11]:
def pcp(X, maxiter=10, k=10): # refactored
    m, n = X.shape
    trans = m<n
    if trans: X = X.T; m, n = X.shape
        
    lamda = 1/np.sqrt(m)
    op_norm = norm_op(X)
    Y = np.copy(X) / max(op_norm, np.linalg.norm( X, np.inf) / lamda)
    mu = k*1.25/op_norm; mu_bar = mu * 1e7; rho = k * 1.5
    
    d_norm = np.linalg.norm(X, 'fro')
    L = np.zeros_like(X); sv = 1
    
    examples = []
    
    for i in range(maxiter):
        print("rank sv:", sv)
        X2 = X + Y/mu
        
        # update estimate of Sparse Matrix by "shrinking/truncating": original - low-rank
        S = shrink(X2 - L, lamda/mu)
        
        # update estimate of Low-rank Matrix by doing truncated SVD of rank sv & reconstructing.
        # count of singular values > 1/mu is returned as svp
        L, svp = svd_reconstruct(X2 - S, sv, 1/mu)
        
        # If svp < sv, you are already calculating enough singular values.
        # If not, add 20% (in this case 240) to sv
        sv = svp + (1 if svp < sv else round(0.05*n))
        
        # residual
        Z = X - L - S
        Y += mu*Z; mu *= rho
        
        examples.extend([S[140,:], L[140,:]])
        
        if m > mu_bar: m = mu_bar
        if converged(Z, d_norm): break
    
    if trans: L=L.T; S=S.T
    return L, S, examples

In [12]:
m, n = M.shape
round(m * .05)

NameError: name 'M' is not defined

In [1]:
import fbpca

ModuleNotFoundError: No module named 'fbpca'